In [1]:
#IMPORTING LIBRARIES
import selenium
import pandas as pd
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
import time
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

1. Write a python program which searches all the product under a particular product from www.amazon.in. 
The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search 
for guitars.
2. In the above question, now scrape the following details of each product listed in first 3 pages of your 
search results and save it in a data frame and csv. In case if any product has less than 3 pages in search 
results then scrape all the products available under that product name. Details to be scraped are: "Brand 
Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and 
“Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

### Intializing the chrome driver and loading amazon site

In [115]:
#Loading the Amazon site
driver = webdriver.Chrome('/Users/akhilmatta/chromedriver')
driver.get('https://www.amazon.in/')
time.sleep(5)

### Serching the product in the site

In [116]:
search_input=driver.find_element_by_xpath('/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[2]/div[1]/input')
search_string = input('Enter the product you want to extract the data for')
search_input.send_keys(search_string)
search_button = driver.find_element_by_xpath('/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[3]/div/span/input')
search_button.click()
feature_url = []
for r in range(1,4):
    product_info = driver.find_elements_by_xpath("//h2[@class='a-size-mini a-spacing-none a-color-base s-line-clamp-2']")
    time.sleep(10)
    for i in product_info:
        site_feature = i.find_element_by_tag_name('a')
        feature_url.append(site_feature.get_attribute('href'))
    nxt_button=driver.find_elements_by_xpath('//span[@class="s-pagination-strip"]/a')
    try:
        driver.get(nxt_button[r].get_attribute('href'))#getting the link from the list for next page
        time.sleep(10)
    except:
        driver.get(nxt_button[r].get_attribute('href'))
        time.sleep(10)


Enter the product you want to extract the data forphones


In [117]:
#creating the empty lists
Brand_name=[]
Name_of_product=[]
Price=[]
Return_exchange=[]
Expected_delivery=[]
Availibility=[]
Product_URL=[]
from tqdm import tqdm
for i in tqdm(feature_url):
    driver.get(i)
    try:
        Brand_name.append(driver.find_element_by_xpath('//*[@id="bylineInfo"]').text)
    except NoSuchElementException:
        Brand_name.append('-')
    try:
        Name_of_product.append(driver.find_element_by_xpath('//*[@id="productTitle"]').text)
    except NoSuchElementException:
        Name_of_product.append('-')       
    try:
        Price.append(driver.find_element_by_xpath("//*[@id='corePrice_desktop']/div/table/tbody/tr[2]/td[2]/span[1]").text)
    except NoSuchElementException as e:
        Price.append('-')
    try:
        Return_exchange.append(driver.find_element_by_xpath('//*[@id="RETURNS_POLICY"]/span/div[2]/a').text)
    except NoSuchElementException as e:
        Return_exchange.append('-')
    try:
        Expected_delivery.append(driver.find_element_by_xpath('//*[@id="mir-layout-DELIVERY_BLOCK-slot-DELIVERY_MESSAGE"]/b').text)
    except NoSuchElementException as e:
        Expected_delivery.append('-')
    try:
        Availibility.append(driver.find_element_by_xpath('//*[@id="availability"]/span').text)
    except NoSuchElementException as e:
        Availibility.append('-')
    Product_URL.append(i)
#driver.close()
        
     

100%|███████████████████████████████████████████| 69/69 [03:53<00:00,  3.39s/it]


### creating a DataFrame

In [118]:
amazon_df = pd.DataFrame({})
amazon_df['Brand Name']=Brand_name
amazon_df['Name Of Product'] = Name_of_product
amazon_df['Price']=Price
amazon_df['Return Exchange']=Return_exchange
amazon_df['Expected Delivery']=Expected_delivery
amazon_df['Availability']=Availibility
amazon_df['Product URL']=Product_URL
amazon_df

,Brand Name,Name Of Product,Price,Return Exchange,Expected Delivery,Availability,Product URL
0,Visit the Samsung Store,"Samsung Galaxy M32 (Light Blue, 4GB RAM, 64GB ...","₹14,999.00",7 Days Replacement,"Tuesday, May 17",In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
1,Visit the Samsung Store,"Samsung Galaxy M53 5G (Mystique Green, 8GB, 12...","₹28,499.00",7 Days Replacement,"Thursday, May 19",In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
2,Brand: realme,"realme narzo 50i (Carbon Black, 4GB RAM+64GB S...","₹8,999.00",7 Days Replacement,"Tuesday, May 17",In stock.,https://www.amazon.in/realme-narzo-Carbon-Blac...
3,Brand: Generic,"A13, 4GB RAM, 64GB Storage Smart Android Mobil...",-,7 Days Replacement,"Friday, May 20",Only 1 left in stock.,https://www.amazon.in/Storage-Smart-Android-Mo...
4,Visit the Oppo Store,"OPPO A31 (Mystery Black, 6GB RAM, 128GB Storag...","₹11,990.00",7 Days Replacement,"Tuesday, May 17",In stock.,https://www.amazon.in/Oppo-Mystery-Storage-Add...
...,...,...,...,...,...,...,...
64,Visit the Redmi Store,"Redmi Note 10T 5G (Chromium White, 6GB RAM, 12...","₹14,999.00",7 Days Replacement,"Tuesday, May 17",In stock.,https://www.amazon.in/Redmi-Note-10T-5G-Dimens...
65,Visit the Xiaomi Store,"Redmi 10A (Charcoal Black, 4GB RAM, 64GB Storage)","₹9,499.00",7 Days Replacement,"Tuesday, May 17",In stock.,https://www.amazon.in/Redmi-Charcoal-Black-64G...
66,Visit the Xiaomi Store,"Redmi 10A (Slate Grey, 4GB RAM, 64GB Storage)","₹9,499.00",7 Days Replacement,"Tuesday, May 17",In stock.,https://www.amazon.in/Redmi-Slate-Grey-64GB-St...
67,Visit the Redmi Store,"Redmi Note 11 (Space Black, 4GB RAM, 64GB Stor...","₹13,499.00",7 Days Replacement,"Tuesday, May 17",In stock.,https://www.amazon.in/gp/slredirect/picassoRed...


Q3: Write a python program to access the search bar and search button on images.google.com and scrape 10
images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

#### Intializing the chrome driver

In [69]:
import os
import urllib
def fetch_images(search_for,no_of_images):
    
    driver = webdriver.Chrome('/Users/akhilmatta/chromedriver')
    url_first_part = 'https://www.google.com.sg/search?q='
    url_second_part = '&source=lnms&tbm=isch&sa=X&ei=0eZEVbj3IJG5uATalICQAQ&ved=0CAcQ_AUoAQ&biw=939&bih=591'
    driver.get(url_first_part+search_for+url_second_part)
    time.sleep(5)
    if not os.path.exists('Scraped_Images'):
        os.mkdir('Scraped_Images')
    parent_tag = driver.find_element_by_id('islmp')
    img_tag = parent_tag.find_elements_by_tag_name('img')
    count=1
    searchitem = []
    srclink = []
    for index,individual_tag in enumerate(img_tag):
        if index < no_of_images:
            src_link = individual_tag.get_attribute('src')
            try:
                if src_link != None:
                    src_link = str(src_link)
                    searchitem.append(search_for)
                    srclink.append(src_link)
    
                    urllib.request.urlretrieve(src_link,os.path.join('Scraped_Images', search_for+str(index)+".jpg"))
                else:
                    raise TypeError
            except Exception as e:
                print(f"failed because of the reason : {e}")
    print('Work Completed Succesfully for : ',search_for)
    driver.close()
    df=pd.DataFrame({"Search Item": searchitem,"src Link":srclink})
    return df
final_df = pd.DataFrame(columns = ['Search Item', 'src Link'])
itms = ['fruits', 'cars', 'Machine Learning', 'Guitar', 'Cakes']
for i in itms:
    itms_df = fetch_images(i,10)
    final_df = pd.concat([final_df, itms_df], ignore_index=True, sort=False)
final_df
        

Work Completed Succesfully for :  fruits
Work Completed Succesfully for :  cars
Work Completed Succesfully for :  Machine Learning
Work Completed Succesfully for :  Guitar
Work Completed Succesfully for :  Cakes


,Search Item,src Link
0,fruits,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
1,fruits,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
2,fruits,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
3,fruits,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
4,fruits,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
5,fruits,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
6,fruits,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
7,fruits,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
8,fruits,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
9,fruits,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."


In [50]:
final_df

,Search Item,src Link


Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on
www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be 
scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, 
“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the 
details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [85]:
driver = webdriver.Chrome(executable_path=r'/Users/akhilmatta/chromedriver', chrome_options=chrome_options)
driver.get('https://www.flipkart.com/')
driver.find_element_by_xpath('/html/body/div[2]/div/div/button').click()
item_search = driver.find_element_by_class_name('_3704LK')
item_search.send_keys('Oneplus Nord')
button= driver.find_element_by_class_name('_34RNph')
button.click()
time.sleep(3)
urls = []
all_products = driver.find_elements_by_xpath("//div[@class='_4ddWXP']")
for product in all_products:
    product_feature = product.find_element_by_tag_name('a')
    urls.append(product_feature.get_attribute('href'))
brand_name = []
title = []
price = []
color = []
display_size = []
rom = []
ram = []
primary_cam = []
battery_capacity = []
product_url = []
for each in urls:
    # Getting the particular page link
    page = driver.get(each)
    try:
        product_title = driver.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[3]/div/div[1]/h1/span')
        brand_name.append(" ".join(product_title.text.split()[:2]))
        title.append(product_title.text)
    except:
        brand_name.append('-')
        title_name.append('-')
    try:
        price.append(driver.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[3]/div/div[3]/div[1]/div/div').text)
    except:
        price.append('-')
    try:
        color.append(driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div/div[1]/table/tbody/tr[4]/td[2]/ul/li').text)
    except:
        color.append('-')
    try:
        driver.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/button').click()
    except:
        pass
    try:
        display_size.append(driver.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[2]/table/tbody/tr[1]/td[2]/ul/li').text)
    except:
        display_size.append('-')
    try:
        rom.append(driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[4]/table/tbody/tr[1]/td[2]/ul/li').text)
    except:
        rom.append('-')
    try:
        ram.append(driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[4]/table/tbody/tr[2]/td[2]/ul/li').text)
    except:
        ram.append('-')
    try:
        primary_cam.append(driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[5]/table/tbody/tr[1]/td[2]/ul/li').text)
    except:
        primary_cam.append('-')
    try:
        battery_capacity.append(driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[7]/table/tbody/tr/td[2]/ul/li').text)
    except:
        battery_capacity.append('-')
    product_url.append(each)
driver.close()
flipkart_data = pd.DataFrame({})
flipkart_data['Brand']=brand_name
flipkart_data['Smartphone name'] = title
flipkart_data['Colour']=color
flipkart_data['RAM']=ram
flipkart_data['Storage(ROM)']=rom
flipkart_data['Primary Camera']=primary_cam
flipkart_data['Display Size']=display_size
flipkart_data['Battery Capacity']=battery_capacity
flipkart_data['Price']=price
flipkart_data['Product URL']=product_url
flipkart_data

,Brand,Smartphone name,Colour,RAM,Storage(ROM),Primary Camera,Display Size,Battery Capacity,Price,Product URL
0,OnePlus Nord,"OnePlus Nord CE 2 Lite 5G (Black Dusk, 128 GB)...",Black Dusk,8 GB,128 GB,64MP Rear Camera,16.74 cm (6.59 inch),5000 mAh,"₹21,999",https://www.flipkart.com/oneplus-nord-ce-2-lit...
1,OnePlus Nord,"OnePlus Nord CE 2 Lite 5G (Black Dusk, 128 GB)...",Black Dusk,6 GB,128 GB,64MP Rear Camera,16.74 cm (6.59 inch),5000 mAh,"₹19,998",https://www.flipkart.com/oneplus-nord-ce-2-lit...
2,OnePlus Nord,"OnePlus Nord 2 5G (Gray Sierra, 128 GB) (8 GB...",Gray Sierra,8 GB,128 GB,50MP Rear Camera,16.33 cm (6.43 inch),4500 mAh,-,https://www.flipkart.com/oneplus-nord-2-5g-gra...
3,OnePlus Nord,"OnePlus Nord 2 5G (Blue Haze, 128 GB) (8 GB RAM)",Blue Haze,8 GB,128 GB,50MP Rear Camera,16.33 cm (6.43 inch),4500 mAh,"₹29,742",https://www.flipkart.com/oneplus-nord-2-5g-blu...
4,NewSelect Back,NewSelect Back Cover for OnePlus Nord (Transp...,-,-,-,-,-,-,₹179,https://www.flipkart.com/newselect-back-cover-...
5,KWINE CASE,KWINE CASE Back Cover for OnePlus Nord CE 2 5G...,-,-,-,-,-,-,-,https://www.flipkart.com/kwine-case-back-cover...
6,Micvir Back,Micvir Back Cover for OnePlus Nord 2 5G (Tran...,-,-,-,-,-,-,-,https://www.flipkart.com/micvir-back-cover-one...
7,GoldKart Back,GoldKart Back Cover for OnePlus Nord CE 2 5G ...,-,-,-,-,-,-,₹229,https://www.flipkart.com/goldkart-back-cover-o...
8,KWINE CASE,KWINE CASE Back Cover for OnePlus Nord CE 2 5G...,-,-,-,-,-,-,₹395,https://www.flipkart.com/kwine-case-back-cover...
9,COSHIBA Edge,COSHIBA Edge To Edge Tempered Glass for Oneplu...,-,-,-,-,-,-,₹175,https://www.flipkart.com/coshiba-edge-tempered...


Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google
maps.

In [94]:
# Intializing the chrome driver
driver = webdriver.Chrome(executable_path=r'/Users/akhilmatta/chromedriver', chrome_options=chrome_options)
time.sleep(3)

# Opening maps web application using chrome driver
url = "https://www.google.com/maps"
driver.get(url)
time.sleep(2)

# Search the country/state/city/dist
inpt = input('Enter country/state/city/dist names to get Latitude and Longitude : ')
search_bar = driver.find_element_by_id('searchboxinput')
search_bar.clear()
time.sleep(2)
search_bar.send_keys(inpt)
search_button = driver.find_element_by_id('searchbox-searchbutton')
search_button.click()
time.sleep(3)
try:
    url = driver.current_url
    latitude_longitude = re.findall(r'@(.*)data',url)
    if len(latitude_longitude):
        lat_long_list = latitude_longitude[0].split(',')
        if len(lat_long_list)>=2:
            latitude = lat_long_list[0]
            longitude = lat_long_list[1]
        print(f'The Latitude and Longitude of {inpt} is {latitude}","{longitude}')
        driver.close()
except:
    print('Error Occured')


Enter country/state/city/dist names to get Latitude and Longitude : karnataka
The Latitude and Longitude of karnataka is 14.9836965","71.8178967


Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) 
from trak.in.

In [111]:
# Intializing the chrome driver
driver = webdriver.Chrome(executable_path=r'/Users/akhilmatta/chromedriver', chrome_options=chrome_options)
time.sleep(3)

#Opening tarck.in website
driver.get('https://trak.in/')
funding_deals_dashboard = driver.find_element_by_id('menu-item-51510')
othr = funding_deals_dashboard.find_element_by_tag_name('a')
driver.get(othr.get_attribute('href'))
driver.maximize_window()

In [112]:
dates = []
strtup_names = []
industries = []
subverticals=[]
city_location=[]
investor_names = []
investment_types = []
amountss = []
for i in range(54,57):
    jan_dates = driver.find_elements_by_xpath(f'//table[@id="tablepress-{i}"]/tbody/tr/td[2]')
    for dat in jan_dates:
        dates.append(dat.text)
    strtup_name = driver.find_elements_by_xpath(f'//table[@id="tablepress-{i}"]/tbody/tr/td[3]/a')
    for names in strtup_name:
        strtup_names.append(names.text)
    industry_names = driver.find_elements_by_xpath(f'//table[@id="tablepress-{i}"]/tbody/tr/td[4]')
    for industry in industry_names:
        industries.append(industry.text)
    sub_vertical_names = driver.find_elements_by_xpath(f'//table[@id="tablepress-{i}"]/tbody/tr/td[5]')
    for sub_vertical in sub_vertical_names:
        subverticals.append(sub_vertical.text)
    city_locs = driver.find_elements_by_xpath(f'//table[@id="tablepress-{i}"]/tbody/tr/td[6]')
    for city in city_locs:
        city_location.append(city.text)
    invstr_names = driver.find_elements_by_xpath(f'//table[@id="tablepress-{i}"]/tbody/tr/td[7]')
    for invstr in invstr_names:
        investor_names.append(invstr.text)
    invst_types = driver.find_elements_by_xpath(f'//table[@id="tablepress-{i}"]/tbody/tr/td[8]')
    for invst_type in invst_types:
        investment_types.append(invst_type.text)
    amounts = driver.find_elements_by_xpath(f'//table[@id="tablepress-{i}"]/tbody/tr/td[9]/strong')
    for amnt in amounts:
        amountss.append(amnt.text)
driver.close()
trakin_df = pd.DataFrame()
trakin_df['Date (dd/mm/yyyy)'] = dates
trakin_df['Startup Name'] = strtup_names
trakin_df['Industry / Vertical'] = industries
trakin_df['Sub-Vertical'] = subverticals
trakin_df['City / Location'] = city_location
trakin_df["Investors' Name"] = investor_names
trakin_df['Investment Type'] = investment_types
trakin_df['Amount (In USD)'] = amountss
trakin_df


,Date (dd/mm/yyyy),Startup Name,Industry / Vertical,Sub-Vertical,City / Location,Investors' Name,Investment Type,Amount (In USD)
0,15/01/2021,Digit Insurance,Financial Services,Insurance Services,Bengaluru,"A91 Partners, Faering Capital, TVS Capital Funds",Venture,"1,80,00,000"
1,28/01/2021,Bombay Shaving Company,Consumer Goods Company,"Shave care, beard care, and skincare products",New Delhi,Reckitt Benckiser,Venture,"6,172,258.50"
2,19/01/2021,DeHaat,AgriTech Startup,online marketplace for farm products and services,Patna,Prosus Ventures,Series C,"30,000,000"
3,19/01/2021,Darwinbox,SaaS,HR Tech,Mumbai,Salesforce Ventures,Seed,"15,000,000"
4,18/01/2021,mfine,Health Tech Startup,AI-powered telemedicine mobile app,Bengaluru,Heritas Capital Management,Venture Round,"16,000,000"
5,18/01/2021,Udayy,EdTech,Online learning platform for kids in class 1-5,Gurgaon,Sequoia Capital,Seed Funding,"15,000,000"
6,11/01/2021,True Elements,Food Startup,Whole Food plant based Nashta,Pune,SIDBI Venture Capital,Series,"100,000,000"
7,13/01/2021,Saveo,B2B E-commerce,Pharmacies,Bengaluru,"Matrix Partners India, RTP Global, others",Seed,"4,000,000"
8,11/02/2021,Doubtnut,Edu Tech,E-Learning Platform,Gurgaon,"SIG Global, Sequoia Capital, WaterBridge Ventu...",Series B,"2,500,000"
9,22/02/2021,Zomato,Hospitality,Online Food Delivery Platform,Gurgaon,"Tiger Global, Kora",Venture,"250,000,000"


Write a program to scrap all the available details of best gaming laptops from digit.in.

In [156]:
# Intializing the chrome driver
driver = webdriver.Chrome(executable_path=r'/Users/akhilmatta/chromedriver', chrome_options=chrome_options)
time.sleep(3)

#Opening tarck.in website
driver.get('https://www.digit.in/')
searchbar_tag = driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div[2]').click()
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div[2]/div/div/input'))).send_keys('best gaming laptops')
wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div[2]/div/div/input'))).send_keys(Keys.RETURN)


In [157]:
product_name = []
market_status = []
release_date = []
official_website = []
os = []
display = []
processor = []
price = []
product_price_details = []
available_at=[]
all_products = driver.find_elements_by_xpath("//div[@class='searchPage']")
get_urls = []
for product in all_products:
    prdct_url = product.find_element_by_tag_name('a')
    get_urls.append(prdct_url.get_attribute('href'))
for url in get_urls:
    driver.get(url)
    product_name.append(driver.find_element_by_xpath('/html/body/div[3]/div[2]/div[2]/h1').text)
    try:
        market_status.append(driver.find_element_by_xpath('/html/body/div[3]/div[2]/div[3]/div[3]/div[1]/div[1]/strong').text)
    except:
        market_status.append('-')
    try:
        release_date.append(driver.find_element_by_xpath('/html/body/div[3]/div[2]/div[3]/div[3]/div[1]/div[2]/strong').text)
    except:
        release_date.append('-')
    try:
        official_website.append(driver.find_element_by_xpath('/html/body/div[3]/div[2]/div[3]/div[3]/div[1]/div[3]/strong').text)
    except:
        official_website.append('-')
    try:
        os.append(driver.find_element_by_xpath('/html/body/div[3]/div[2]/div[3]/div[3]/div[3]/div/ul/li[1]/div/p[2]/strong').text)
    except:
        os.append('-')
    try:
        display.append(driver.find_element_by_xpath('/html/body/div[3]/div[2]/div[3]/div[3]/div[3]/div/ul/li[2]/div/p[2]/strong').text)
    except:
        display.append('-')
    try:
        processor.append(driver.find_element_by_xpath('/html/body/div[3]/div[2]/div[3]/div[3]/div[3]/div/ul/li[3]/div/p[2]/strong').text)
    except:
        processor.append('-')
    try:
        price.append(driver.find_element_by_xpath('/html/body/div[3]/div[2]/div[3]/div[3]/div[4]/div/h2/strong').text)
    except:
        price.append('-')
    try:
        product_price_details.append(driver.find_element_by_xpath('/html/body/div[3]/div[2]/div[7]/div[2]/p').text)
    except:
        product_price_details.append('-')
    try:
        dta = driver.find_element_by_xpath("//div[@class='price']/span")
        avil_url = dta.get_attribute('url')
        available_at.append(avil_url)
    except:
        available_at.append('-')
driver.close()
digit_df = pd.DataFrame()
digit_df['Product Name'] = product_name
digit_df['Market Status'] = market_status
digit_df['Release Date'] = release_date
digit_df['Official Website'] = official_website
digit_df['OS'] = os
digit_df["Display"] = display
digit_df['Processor'] = processor
digit_df['Amount'] = price
digit_df['Product Price Comparision'] = product_price_details
digit_df['Available At'] = available_at
digit_df        
    
    

,Product Name,Market Status,Release Date,Official Website,OS,Display,Processor,Amount,Product Price Comparision,Available At
0,Asus G750JX-CV069P,LAUNCHED,"14 Oct, 2014",Asus,NA,NA,NA,"₹ 149,999",Asus G750JX-CV069P price in India starts from ...,http://dl.flipkart.com/dl/asus-g750jx-cv069p-l...
1,Lenovo G50-30,LAUNCHED,"17 Mar, 2016",Lenovo,NA,NA,NA,"₹ 25,694",Lenovo G50-30 price in India starts from ₹ 256...,https://www.amazon.in/dp/B00W537OQI
2,ASUS G53J - A 3D gamer's delight but we want more,LAUNCHED,"13 Jan, 2011",Asus,NA,NA,NA,"₹ 112,499",-,None
3,Dell Vostro 15 3500,LAUNCHED,"17 Mar, 2021",Dell,Windows 10 Home,"15.6"" (1920 X 1080)",11th Gen Intel® Core™ I3-1115G4 | NA,"₹ 38,489",-,None
4,HP Envy 15-k006tx,LAUNCHED,"14 Oct, 2014",HP,NA,NA,NA,"₹ 80,510",-,None
5,HP Envy dv6-7206tx notebook,LAUNCHED,"23 Feb, 2013",HP,Windows 8 64 Bit,"15.6"" (1366 X 768)",Intel Core I7 (3rd Generation) | 2.4 Ghz,"₹ 65,705",-,None
6,Samsung Series 9 (NP-900X3C),DISCONTINUED,"01 Nov, 2012",Samsung,NA,NA,NA,-,-,None
7,Asus GTX980 Matrix Platinum,LAUNCHED,"30 Jan, 2015",Asus,-,-,-,-,-,None
8,Dell 15 (2021) i5-10200H,LAUNCHED,"11 Apr, 2022",Dell,NA,NA,NA,"₹ 82,635",Dell 15 (2021) i5-10200H price in India starts...,https://www.amazon.in/dp/B095S8L3Z7
9,Lenovo IdeaPad Y570,LAUNCHED,"07 Feb, 2012",Lenovo,Windows 7 (64-Bit),"15.6"" (1366 X 768)",Intel Core I7 (2nd Generation) | 2 Ghz,"₹ 49,900",-,None


Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be 
scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [38]:
# Intializing the chrome driver
driver = webdriver.Chrome(executable_path=r'/Users/akhilmatta/chromedriver', chrome_options=chrome_options)
time.sleep(3)

#Opening tarck.in website
driver.get('https://www.forbes.com')
time.sleep(3)
driver.find_element_by_xpath('/html/body/div[1]/header/nav/div[1]/button[1]').click()
billonaire_tag = driver.find_element_by_xpath('/html/body/div[1]/header/nav/div[3]/ul/li[1]/div[2]/ul/li[2]/a')
billionarie_href = billonaire_tag.get_attribute('href')
driver.get(billionarie_href)
time.sleep(5)
rows = driver.find_elements_by_xpath("//div[@class='table-row-group__container']")
ranks = []
person_names = []
persons_net_worth = []
ages = []
citizen_ships = []
sources = []
industries = []
for row in rows:
    persn_ranks = row.find_elements_by_class_name('rank')
    persn_names = row.find_elements_by_class_name('personName') 
    persn_networth = row.find_elements_by_class_name('netWorth')
    person_ages = row.find_elements_by_class_name('age')
    person_citizenship = row.find_elements_by_class_name('countryOfCitizenship')
    person_source = row.find_elements_by_class_name('source')
    person_industry = row.find_elements_by_class_name('category')
    [person_names.append(person.text) for person in persn_names]
    [ranks.append(rank.text[:-1]) for rank in persn_ranks]
    [persons_net_worth.append(net_worth.text) for net_worth in persn_networth]
    [ages.append(age.text) for age in person_ages]
    [citizen_ships.append(citizenship.text) for citizenship in person_citizenship]
    [sources.append(source.text) for source in person_source]
    [industries.append(industry.text) for industry in person_industry]
    
driver.close()
forbes_df = pd.DataFrame()
forbes_df['Rank'] = ranks
forbes_df['Person Name'] = person_names
forbes_df['Person Net Worth'] = persons_net_worth
forbes_df['Age'] = ages
forbes_df['CitizenShip'] = citizen_ships
forbes_df["Source"] = sources
forbes_df['Industry'] = industries
forbes_df 

,Rank,Person Name,Person Net Worth,Age,CitizenShip,Source,Industry
0,1,Elon Musk,$219 B,50,United States,"Tesla, SpaceX",Automotive
1,2,Jeff Bezos,$171 B,58,United States,Amazon,Technology
2,3,Bernard Arnault & family,$158 B,73,France,LVMH,Fashion & Retail
3,4,Bill Gates,$129 B,66,United States,Microsoft,Technology
4,5,Warren Buffett,$118 B,91,United States,Berkshire Hathaway,Finance & Investments
...,...,...,...,...,...,...,...
195,192,Marcel Herrmann Telles,$10.3 B,72,Brazil,beer,Food & Beverage
196,197,Leon Black,$10 B,70,United States,private equity,Finance & Investments
197,197,Joe Gebbia,$10 B,40,United States,Airbnb,Technology
198,197,David Geffen,$10 B,79,United States,"movies, record labels",Media & Entertainment


Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted 
from any YouTube Video.

In [96]:
# Intializing the chrome driver
driver = webdriver.Chrome(executable_path=r'/Users/akhilmatta/chromedriver', chrome_options=chrome_options)
time.sleep(3)

#Opening tarck.in website
driver.get('https://www.youtube.com/watch?v=AJtDXIazrMo')
time.sleep(3)

while True:
    scroll_height = 4000
    document_height_before = driver.execute_script("return document.documentElement.scrollHeight")
    driver.execute_script(f"window.scrollTo(0, {document_height_before + scroll_height});")
    time.sleep(1.5)
    document_height_after = driver.execute_script("return document.documentElement.scrollHeight")
    if document_height_after == document_height_before:
        break
comments = driver.find_elements_by_xpath('//*[@id="content-text"]')
dtime = driver.find_elements_by_xpath('//*[@id="header-author"]/yt-formatted-string/a')
votes = driver.find_elements_by_xpath('//*[@id="vote-count-middle"]')
cmnts=[]
dtetime = []
upvotes = []
[cmnts.append(i.text) for i in comments]
[dtetime.append(j.text) for j in dtime]
[upvotes.append(k.text) for k in votes]
driver.close()
youtube_cmnts_df = pd.DataFrame()
youtube_cmnts_df['Video Comments'] = cmnts[0:500]
youtube_cmnts_df['Comment Date Time'] = dtetime[0:500]
youtube_cmnts_df['Comment Votes'] = dtetime[0:500]
youtube_cmnts_df

,Video Comments,Comment Date Time,Comment Votes
0,"6 years later, if you're still listening to th...",7 months ago (edited),7 months ago (edited)
1,"It's May 2022, let's see how many legends are ...",11 days ago,11 days ago
2,It's been 7 years but still this song hits me ...,3 weeks ago,3 weeks ago
3,"You're the light, you're the night\nYou're the...",2 weeks ago,2 weeks ago
4,"The sound of this song just never gets boring,...",1 month ago,1 month ago
...,...,...,...
495,2012~2022 i still heard this music ;),1 day ago,1 day ago
496,This song hits so Different 🥲,3 weeks ago,3 weeks ago
497,After 6 years I knew that why this song has 2....,1 year ago,1 year ago
498,Most trending song ever and most Favorite 💕💕,1 month ago,1 month ago


Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in 
“London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, 
overall reviews, privates from price, dorms from price, facilities and property description

In [21]:
driver=webdriver.Chrome(r'/Users/akhilmatta/chromedriver')
driver.get('https://www.hostelworld.com/s?q=London,%20England&country=England&city=London&type=city&id=3&from=2021-10-25&to=2021-10-28&guests=2&page=1')
time.sleep(5)

In [31]:
hostel=[]
dist=[]
rating=[]
overallrev=[]

#scrapping hostel names 
hostel_name=driver.find_elements_by_xpath('//div[@class="property-card"]/div[2]/a[1]/div[1]/h2')
[hostel.append(i.text) for i in hostel_name]   
hostels=driver.find_elements_by_xpath('//h2[@class="title title-6"]/a')
[hostel.append(i.text) for i in hostels]    
# Scraping distances
distance=driver.find_elements_by_xpath('//div[@class="subtitle body-3"]')
[dist.append(i.text) for i in distance]   
#scrapping ratings
ratings=driver.find_elements_by_xpath('//div[@class="score orange small"]')
[rating.append(i.text) for i in ratings]    
try:
    other_ratings=driver.find_elements_by_xpath('//div[@class="rating rating-summary-container big"]/div[1]')
    [rating.append(i.text) for i in other_ratings]        
except:
    rating.append('_')
#scrapping Overall reviews

overall_reviews=driver.find_elements_by_xpath('//div[@class="keyword"]')
[overallrev.append(i.text) for i in overall_reviews]    
hostels=pd.DataFrame({})
hostels['Name']=hostel
hostels['Distance']=dist
hostels['Ratings']=rating
hostels['Overall Reviews']=overallrev
hostels


,Name,Distance,Ratings,Overall Reviews
0,St Christopher's Village,Hostel - 1.8km from city centre,8.1,Fabulous
1,Palmers Lodge - Swiss Cottage,Hostel - 6.5km from city centre,9.0,Superb
2,Selina Camden,Hostel - 5.5km from city centre,9.3,Superb
3,St Christopher's Village,Hostel - 1.8km from city centre Show on Map,8.1,Fabulous
4,Palmers Lodge - Swiss Cottage,Hostel - 6.5km from city centre Show on Map,9.0,Superb
5,Selina Camden,Hostel - 5.5km from city centre Show on Map,9.3,Superb
6,Generator London,Hostel - 3km from city centre Show on Map,7.7,Very Good
7,Astor Museum Inn,Hostel - 2.2km from city centre Show on Map,8.6,Fabulous
8,Urbany Hostel London,Hostel - 5.4km from city centre Show on Map,9.4,Superb
9,Astor Hyde Park,Hostel - 4.3km from city centre Show on Map,8.5,Fabulous
